In [7]:
import requests
import json
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def request_vision_api(image_path):
    endpoint = "https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
    api_key = "ee6d3d3de333480291b572fc7147b015"
    
    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }
    params = {
        "api-version": "2024-02-01",
        "features": "read, caption"
    }
   
    with open(image_path, 'rb') as image:
        image_data = image.read()
        
    response = requests.post(endpoint,
                             headers=headers,
                             params=params,
                             data=image_data)
    
    response_json = response.json()
    return response_json

def change_image(image_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # 폰트 설정
    try:
        font = ImageFont.truetype("arial.ttf", size=20)
    except IOError:
        font = ImageFont.load_default()
    
    response_json = request_vision_api(image_path=image_path)
    if "readResult" in response_json and "blocks" in response_json["readResult"]:
        block_list = response_json["readResult"]["blocks"]
        
        for block in block_list:
            line_list = block["lines"]
            for line in line_list:
                text = line["text"]
                bounding_polygon = line["boundingPolygon"]
                if len(bounding_polygon) >= 4:
                    polygon = [(p['x'], p['y']) for p in bounding_polygon]
                    draw.polygon(polygon, outline='red', fill=None, width=3)
                    draw.text((bounding_polygon[3]['x'], bounding_polygon[3]['y'] + 3), text=text, fill='green', font=font)
                    print(text, bounding_polygon)
    
    return image

import gradio as gr

with gr.Blocks() as demo:
    input_image = gr.Image(label="업로드", type="filepath", width=800)
    output_image = gr.Image(label="결과", type="pil", width=800)

    input_image.change(fn=change_image, inputs=input_image, outputs=output_image)

demo.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 396, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\USER\AppData\Loc